In [1]:
# Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

# Prerequisites
#import sys
#!{sys.executable} -m pip install scikit-learn folium pygeohash catboost xgboost lightgbm

# Load Dependencies
import time
from sklearn.pipeline import Pipeline
import pickle
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import xgboost as xgb

#import folium.plugins as plugins

from math import sqrt
from sklearn import metrics
from sklearn.svm import SVR
#from folium.plugins import HeatMap
#from IPython.display import display
from catboost import CatBoostRegressor
from scipy.stats import boxcox, zscore
from statsmodels.formula.api import ols
#from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures

# Display
%matplotlib inline
sns.set(font_scale=1)
sns.set_style("white")


import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/df-train_set.csv")
test = pd.read_csv('https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/df-test_set.csv')

In [4]:
train=train[train['Commodities'].str.contains('APPLE GOLDEN DELICIOUS')]
test=test[test['Commodities'].str.contains('APPLE GOLDEN DELICIOUS')]

In [5]:
test.set_index('Index', inplace=True)

In [6]:
index = test.index
index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            676, 677, 678, 679, 680, 681, 682, 683, 684, 685],
           dtype='int64', name='Index', length=685)

In [7]:
train['Date'] = pd.to_datetime(train['Date'])
train.dtypes

Province                    object
Container                   object
Size_Grade                  object
Weight_Kg                  float64
Commodities                 object
Date                datetime64[ns]
Low_Price                  float64
High_Price                 float64
Sales_Total                float64
Total_Qty_Sold               int64
Total_Kg_Sold              float64
Stock_On_Hand                int64
avg_price_per_kg           float64
dtype: object

In [8]:
train["month"] = train["Date"].dt.month
train.loc[train['month'] == 8,'season'] ='spring'
train.loc[train['month'] == 9,'season'] ='spring'
train.loc[train['month'] == 10,'season'] ='spring'
train.loc[train['month'] == 12,'season'] ='summer'
train.loc[train['month'] == 1,'season'] ='summer'
train.loc[train['month'] == 2,'season'] ='summer'
train.loc[train['month'] == 5,'season'] ='autumn'
train.loc[train['month'] == 4,'season'] ='autumn'
train.loc[train['month'] == 3,'season'] ='autumn'
train.loc[train['month'] == 6,'season'] ='winter'
train.loc[train['month'] == 8,'season'] ='winter'
train.loc[train['month'] == 7,'season'] ='winter'
train

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg,month,season
1,CAPE,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-09-09,150.0,170.0,51710.0,332,6075.6,822,8.51,9,spring
7,CAPE,JG110,2M,11.0,APPLE GOLDEN DELICIOUS,2020-04-14,50.0,50.0,16000.0,320,3520.0,0,4.55,4,autumn
24,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,APPLE GOLDEN DELICIOUS,2020-04-16,55.0,55.0,990.0,18,162.0,1506,6.11,4,autumn
40,CAPE,M4183,1S,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,80.0,120.0,32020.0,388,7100.4,443,4.51,5,autumn
69,EASTERN CAPE,IA400,1S,400.0,APPLE GOLDEN DELICIOUS,2020-09-28,1800.0,1800.0,1800.0,1,400.0,2,4.50,9,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64276,NATAL,EC120,1S,12.0,APPLE GOLDEN DELICIOUS,2020-03-04,96.0,96.0,1920.0,20,240.0,53,8.00,3,autumn
64291,ORANGE FREE STATE,M4183,1X,18.3,APPLE GOLDEN DELICIOUS,2020-02-22,130.0,130.0,7280.0,56,1024.8,74,7.10,2,summer
64297,CAPE,JE090,2M,9.0,APPLE GOLDEN DELICIOUS,2020-04-22,50.0,60.0,4540.0,89,801.0,793,5.67,4,autumn
64304,CAPE,JG110,2M,11.0,APPLE GOLDEN DELICIOUS,2020-03-04,30.0,40.0,2140.0,70,770.0,0,2.78,3,autumn


In [9]:
test['Date'] = pd.to_datetime(test['Date'])
test.dtypes

Province                  object
Container                 object
Size_Grade                object
Weight_Kg                float64
Commodities               object
Date              datetime64[ns]
Low_Price                float64
High_Price               float64
Sales_Total              float64
Total_Qty_Sold             int64
Total_Kg_Sold            float64
Stock_On_Hand              int64
dtype: object

In [10]:
test["month"] = test["Date"].dt.month
test.loc[test['month'] == 11,'season'] ='spring'
test.loc[test['month'] == 10,'season'] ='spring'
test.loc[test['month'] == 9,'season'] ='spring'
test.loc[test['month'] == 1,'season'] ='summer'
test.loc[test['month'] == 2,'season'] ='summer'
test.loc[test['month'] == 12,'season'] ='summer'
test.loc[test['month'] == 5,'season'] ='autumn'
test.loc[test['month'] == 4,'season'] ='autumn'
test.loc[test['month'] == 3,'season'] ='autumn'
test.loc[test['month'] == 8,'season'] ='winter'
test.loc[test['month'] == 7,'season'] ='winter'
test.loc[test['month'] == 6,'season'] ='winter'
test

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,month,season
Index,,,,,,,,,,,,,,
1,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-07-09,128.0,136.0,5008.0,38,456.0,0,7,winter
2,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,APPLE GOLDEN DELICIOUS,2020-01-20,220.0,220.0,1760.0,8,146.4,2,1,summer
3,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,APPLE GOLDEN DELICIOUS,2020-08-19,120.0,120.0,720.0,6,72.0,45,8,winter
4,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,APPLE GOLDEN DELICIOUS,2020-05-06,160.0,160.0,160.0,1,18.3,8,5,autumn
5,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,140.0,160.0,14140.0,100,1830.0,19,5,autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,APPLE GOLDEN DELICIOUS,2020-05-26,30.0,30.0,1320.0,44,396.0,856,5,autumn
682,CAPE,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-01-10,50.0,132.0,6290.0,52,624.0,785,1,summer
683,W.CAPE-BERGRIVER ETC,JE090,2X,9.0,APPLE GOLDEN DELICIOUS,2020-06-03,55.0,55.0,220.0,4,36.0,170,6,winter


In [11]:
train = pd.get_dummies(train, columns=['Province', 'Container', 'Size_Grade','Commodities', 'season'],drop_first = True)
test= pd.get_dummies(test, columns=['Province', 'Container', 'Size_Grade','Commodities', 'season'],drop_first = True)
train['avg_price_per_kg'] = train.pop('avg_price_per_kg')
train= train.drop('Date',  axis=1)
test= test.drop('Date',  axis=1)
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values
X_test = test.iloc[:,:].values


In [12]:
train

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,month,Province_EASTERN CAPE,Province_NATAL,...,Size_Grade_1X,Size_Grade_2L,Size_Grade_2M,Size_Grade_2S,Size_Grade_2U,Size_Grade_2X,season_spring,season_summer,season_winter,avg_price_per_kg
1,18.3,150.0,170.0,51710.0,332,6075.6,822,9,0,0,...,0,0,0,0,0,0,1,0,0,8.51
7,11.0,50.0,50.0,16000.0,320,3520.0,0,4,0,0,...,0,0,1,0,0,0,0,0,0,4.55
24,9.0,55.0,55.0,990.0,18,162.0,1506,4,0,0,...,0,0,0,1,0,0,0,0,0,6.11
40,18.3,80.0,120.0,32020.0,388,7100.4,443,5,0,0,...,0,0,0,0,0,0,0,0,0,4.51
69,400.0,1800.0,1800.0,1800.0,1,400.0,2,9,1,0,...,0,0,0,0,0,0,1,0,0,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64276,12.0,96.0,96.0,1920.0,20,240.0,53,3,0,1,...,0,0,0,0,0,0,0,0,0,8.00
64291,18.3,130.0,130.0,7280.0,56,1024.8,74,2,0,0,...,1,0,0,0,0,0,0,1,0,7.10
64297,9.0,50.0,60.0,4540.0,89,801.0,793,4,0,0,...,0,0,1,0,0,0,0,0,0,5.67
64304,11.0,30.0,40.0,2140.0,70,770.0,0,3,0,0,...,0,0,1,0,0,0,0,0,0,2.78


In [13]:
test

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,month,Province_EASTERN CAPE,Province_NATAL,...,Size_Grade_1U,Size_Grade_1X,Size_Grade_2L,Size_Grade_2M,Size_Grade_2S,Size_Grade_2U,Size_Grade_2X,season_spring,season_summer,season_winter
Index,,,,,,,,,,,,,,,,,,,,,
1,12.0,128.0,136.0,5008.0,38,456.0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,1
2,18.3,220.0,220.0,1760.0,8,146.4,2,1,0,0,...,0,1,0,0,0,0,0,0,1,0
3,12.0,120.0,120.0,720.0,6,72.0,45,8,0,0,...,0,0,0,0,0,0,0,0,0,1
4,18.3,160.0,160.0,160.0,1,18.3,8,5,0,0,...,0,0,0,0,0,0,0,0,0,0
5,18.3,140.0,160.0,14140.0,100,1830.0,19,5,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,9.0,30.0,30.0,1320.0,44,396.0,856,5,0,0,...,0,0,0,0,1,0,0,0,0,0
682,12.0,50.0,132.0,6290.0,52,624.0,785,1,0,0,...,0,0,0,0,0,0,0,0,1,0
683,9.0,55.0,55.0,220.0,4,36.0,170,6,0,0,...,0,0,0,0,0,0,1,0,0,1


In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print("X_train:", X_train[0,0:11], "... Training")
print("y_train:", y_train[0])
print("X_val:", X_val[0,0:11], "... Validation")
print("y_val:", y_val[0])
print("X_test:", X_test[0,0:11], "... Testing")

X_train: [1.83000e+01 1.40000e+02 1.60000e+02 1.01160e+05 6.69000e+02 1.22427e+04
 0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00] ... Training
y_train: 8.26
X_val: [1.8300e+01 1.8000e+02 2.0000e+02 1.2620e+04 6.8000e+01 1.2444e+03
 2.7500e+02 3.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00] ... Validation
y_val: 10.14
X_test: [  12.  128.  136. 5008.   38.  456.    0.    7.    0.    0.    0.] ... Testing


In [15]:
# Make new copies for scaling
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

# Apply scaler to numeric features
sc = StandardScaler()
X_train_scaled[:,4:] = sc.fit_transform(X_train[:,4:])
X_val_scaled[:,4:] = sc.transform(X_val[:,4:])
X_test_scaled[:,4:] = sc.transform(X_test[:,4:])

In [16]:
# Convert to dataframes
all_columns = ['Weight_Kg','Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold',
       'Total_Kg_Sold', 'Stock_On_Hand', 'month', 'Province_EASTERN CAPE',
       'Province_NATAL', 'Province_ORANGE FREE STATE', 'Province_TRANSVAAL',
       'Province_W.CAPE-BERGRIVER ETC', 'Province_WEST COAST',
       'Container_DT063', 'Container_EC120', 'Container_EF120',
       'Container_EG140', 'Container_IA400', 'Container_JE090',
       'Container_JG110', 'Container_M4183', 'Container_M6125',
       'Container_M9125', 'Size_Grade_1M', 'Size_Grade_1S', 'Size_Grade_1U',
       'Size_Grade_1X', 'Size_Grade_2L', 'Size_Grade_2M', 'Size_Grade_2S',
       'Size_Grade_2U', 'Size_Grade_2X', 'season_spring', 'season_summer', 'season_winter' ]
# Not scaled
X_train_df = pd.DataFrame(X_train,columns=all_columns)
X_val_df = pd.DataFrame(X_val,columns=all_columns)
X_test_df = pd.DataFrame(X_test,columns=all_columns)
X_all_df = pd.DataFrame(X,columns=all_columns)
# Scaled
X_train_scaled_df = pd.DataFrame(X_train_scaled,columns=all_columns)
X_val_scaled_df = pd.DataFrame(X_val_scaled,columns=all_columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled,columns=all_columns)
# Convert to numeric
X_train_scaled_df = X_train_scaled_df.apply(pd.to_numeric)
X_val_scaled_df = X_val_scaled_df.apply(pd.to_numeric)
X_test_scaled_df = X_test_scaled_df.apply(pd.to_numeric)
X_train_df = X_train_df.apply(pd.to_numeric)
X_val_df = X_val_df.apply(pd.to_numeric)
X_test_df = X_test_df.apply(pd.to_numeric)
X_all_df = X_all_df.apply(pd.to_numeric)

X_train_scaled_df.head()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,month,Province_EASTERN CAPE,Province_NATAL,...,Size_Grade_1U,Size_Grade_1X,Size_Grade_2L,Size_Grade_2M,Size_Grade_2S,Size_Grade_2U,Size_Grade_2X,season_spring,season_summer,season_winter
0,18.3,140.0,160.0,101160.0,1.598354,1.484784,-0.567551,-1.713083,-0.348322,-0.247037,...,-0.02387,-0.30226,-0.254731,-0.368557,-0.436473,-0.092821,-0.115203,-0.310048,2.130594,-0.850446
1,9.0,40.0,60.0,1920.0,-0.413907,-0.412285,-0.567551,0.521539,2.870907,-0.247037,...,-0.02387,-0.30226,-0.254731,2.713283,-0.436473,-0.092821,-0.115203,-0.310048,-0.469353,1.175854
2,18.3,90.0,90.0,360.0,-0.553018,-0.468428,-0.567551,0.893976,-0.348322,-0.247037,...,-0.02387,3.30841,-0.254731,-0.368557,-0.436473,-0.092821,-0.115203,-0.310048,-0.469353,1.175854
3,12.0,120.0,120.0,240.0,-0.559489,-0.476324,-0.392049,0.893976,-0.348322,-0.247037,...,-0.02387,-0.30226,-0.254731,-0.368557,-0.436473,-0.092821,-0.115203,-0.310048,-0.469353,1.175854
4,18.3,100.0,140.0,3440.0,-0.455964,-0.380313,-0.301556,0.893976,-0.348322,-0.247037,...,-0.02387,-0.30226,-0.254731,-0.368557,-0.436473,-0.092821,-0.115203,-0.310048,-0.469353,1.175854


In [17]:
# Final features used in model
features =  ['Weight_Kg', 'Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold',
       'Total_Kg_Sold', 'Stock_On_Hand', 'Province_EASTERN CAPE',
       'Province_NATAL', 'Province_ORANGE FREE STATE', 'Province_TRANSVAAL',
       'Province_W.CAPE-BERGRIVER ETC', 'Province_WEST COAST',
       'Container_DT063', 'Container_EC120', 'Container_EF120',
       'Container_EG140', 'Container_IA400', 'Container_JE090',
       'Container_JG110', 'Container_M4183', 'Container_M6125',
       'Container_M9125', 'Size_Grade_1M', 'Size_Grade_1S', 'Size_Grade_1U',
       'Size_Grade_1X', 'Size_Grade_2L', 'Size_Grade_2M', 'Size_Grade_2S',
       'Size_Grade_2U', 'Size_Grade_2X', 'season_spring', 'season_summer', 'season_winter']
X_all = pd.DataFrame(X_all_df.loc[:,features],columns=features).values
X_train = pd.DataFrame(X_train_df.loc[:,features],columns=features).values
X_val = pd.DataFrame(X_val_df.loc[:,features],columns=features).values
X_test = pd.DataFrame(X_test_df.loc[:,features],columns=features).values
X_train_scaled = pd.DataFrame(X_train_scaled_df.loc[:,features],columns=features).values
X_val_scaled = pd.DataFrame(X_val_scaled_df.loc[:,features],columns=features).values

In [18]:
modelstart= time.time()
cb_model = CatBoostRegressor(iterations=700,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 0,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
cb_model.fit(X_train, y_train, eval_set=(X_val,y_val), use_best_model=True,verbose=True)
y_pred_val = cb_model.predict(X_val)
cb_model_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred_val))
cb_model_r2 = metrics.r2_score(y_val, y_pred_val)
print('RMSE:', cb_model_rmse)
print('R-squared:', cb_model_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)))

0:	learn: 2.2128071	test: 2.2838512	best: 2.2838512 (0)	total: 232ms	remaining: 2m 41s
75:	learn: 1.0226539	test: 1.0552092	best: 1.0552092 (75)	total: 7.26s	remaining: 59.6s
150:	learn: 0.6740095	test: 0.7309334	best: 0.7309334 (150)	total: 15.3s	remaining: 55.7s
225:	learn: 0.5342969	test: 0.6267714	best: 0.6267714 (225)	total: 22.4s	remaining: 46.9s
300:	learn: 0.4504435	test: 0.5736269	best: 0.5736269 (300)	total: 30.5s	remaining: 40.4s
375:	learn: 0.3938633	test: 0.5462419	best: 0.5462419 (375)	total: 37.8s	remaining: 32.6s
450:	learn: 0.3540176	test: 0.5279676	best: 0.5279676 (450)	total: 46.1s	remaining: 25.4s
525:	learn: 0.3242207	test: 0.5145449	best: 0.5145449 (525)	total: 56.5s	remaining: 18.7s
600:	learn: 0.2901455	test: 0.5024224	best: 0.5024224 (600)	total: 1m 6s	remaining: 11s
675:	learn: 0.2609486	test: 0.4932316	best: 0.4932316 (675)	total: 1m 16s	remaining: 2.72s
699:	learn: 0.2526932	test: 0.4907605	best: 0.4907605 (699)	total: 1m 19s	remaining: 0us

bestTest = 0.490

In [19]:
# Make submission file
final_model = CatBoostRegressor(iterations=700,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 0,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
final_model.fit(X_all,y)
y_test = final_model.predict(X_test)
results = pd.DataFrame({"Index":index,"avg_price_per_kg": y_test})
results.to_csv("catbooster5", index=False)

0:	learn: 2.2196918	total: 169ms	remaining: 1m 57s
75:	learn: 1.0127051	total: 8.7s	remaining: 1m 11s
150:	learn: 0.6609415	total: 18.6s	remaining: 1m 7s
225:	learn: 0.5211886	total: 28.1s	remaining: 58.9s
300:	learn: 0.4454253	total: 37.6s	remaining: 49.8s
375:	learn: 0.3926379	total: 47.3s	remaining: 40.8s
450:	learn: 0.3545476	total: 58s	remaining: 32s
525:	learn: 0.3252763	total: 1m 7s	remaining: 22.4s
600:	learn: 0.2981201	total: 1m 17s	remaining: 12.8s
675:	learn: 0.2716256	total: 1m 27s	remaining: 3.1s
699:	learn: 0.2638811	total: 1m 29s	remaining: 0us


In [20]:
modelstart= time.time()
final_model = xgb.XGBRegressor(max_features= 5, min_samples_leaf=1, min_samples_split= 2, n_estimators= 300)
final_model.fit(X_train,y_train)
y_pred_val_final = final_model.predict(X_val)
y_pred_test_final = final_model.predict(X_test)
final_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred_val_final))
final_r2 = metrics.r2_score(y_val, y_pred_val_final)
print("Testing: LightGBM")
print('RMSE:', final_rmse)
print('R-squared:', final_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)/60))

[17:18:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Testing: LightGBM
RMSE: 0.37516228402589336
R-squared: 0.9736392056740926
Model Runtime: 0.02 seconds


In [21]:
# Make submission file
final_model =  xgb.XGBRegressor(max_features= 5, min_samples_leaf=1, min_samples_split= 2, n_estimators= 300)
final_model.fit(X_all,y)
y_test = final_model.predict(X_test)
results1 = pd.DataFrame({"Index":index,"avg_price_per_kg": y_test})
results1.to_csv("xgbooster", index=False)

[17:18:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [22]:
modelstart= time.time()
xg_reg = GradientBoostingRegressor()
xg_reg.fit(X_train,y_train)
y_pred = xg_reg.predict(X_val)
xg_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred))
xg_r2 = metrics.r2_score(y_val, y_pred)

print("Testing: XGBoost")
print('RMSE:', xg_rmse)
print('R-squared:', xg_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)))

Testing: XGBoost
RMSE: 0.4651226366221443
R-squared: 0.9594813373400253
Model Runtime: 0.36 seconds


In [23]:
# Make submission file
final_model = GradientBoostingRegressor()
final_model.fit(X_all,y)
y_test = final_model.predict(X_test)
results2 = pd.DataFrame({"Index":index,"Price": y_test})
results2.to_csv("Gbooster", index=False)

In [24]:
RF = RandomForestRegressor(n_estimators=100, max_depth=5)
RF.fit(X_train,y_train)
# Get predictions
y_pred = RF.predict(X_val)
RF_r2 = metrics.r2_score(y_val,y_pred)
print("Testing: RandomForest")
print("RMSE:",np.sqrt(mean_squared_error(y_val,y_pred)))
print('R-squared:', RF_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)))

Testing: RandomForest
RMSE: 0.7553990937033949
R-squared: 0.893125752696243
Model Runtime: 7.96 seconds


In [25]:
# Create LASSO model object, setting alpha to 0.01
lasso = Lasso(alpha=0.01)
# Train the LASSO model
lasso.fit(X_train, y_train)

Lasso(alpha=0.01)

In [26]:
# Least squares using non-zero variables from LASSO
lm_subset = LinearRegression()
# Least squares using all predictors
lm_all = LinearRegression()
# Ridge using all predictors
ridge = Ridge()

lm_subset.fit(X_train, y_train)
lm_all.fit(X_train, y_train)
ridge.fit(X_train, y_train)
# Make training set predictions for each model
train_lm_subset = lm_subset.predict(X_train)
train_lm_all = lm_all.predict(X_train)
train_ridge = ridge.predict(X_train)
train_lasso = lasso.predict(X_train)

In [27]:
# Make test set predictions for each model
test_lm_subset = lm_subset.predict(X_val)
test_lm_all = lm_all.predict(X_val)
test_ridge = ridge.predict(X_val)
test_lasso = lasso.predict(X_val)

In [28]:
# Dictionary of results
results_dict = {'Training RMSE':
                    {
                        "Least Squares, Subset": np.sqrt(mean_squared_error(y_train, train_lm_subset)),
                        "Least Squares, All": np.sqrt(mean_squared_error(y_train, train_lm_all)),
                        "Ridge": np.sqrt(mean_squared_error(y_train, train_ridge)),
                        "LASSO": np.sqrt(mean_squared_error(y_train, train_lasso))
                    },
                    'Test RMSE':
                    {
                        "Least Squares, Subset": np.sqrt(mean_squared_error(y_val, test_lm_subset)),
                        "Least Squares, All": np.sqrt(mean_squared_error(y_val, test_lm_all)),
                        "Ridge": np.sqrt(mean_squared_error(y_val, test_ridge)),
                        "LASSO": np.sqrt(mean_squared_error(y_val, test_lasso))
                    }
                }
# Create dataframe from dictionary
results_df = pd.DataFrame(data=results_dict)
# View the results
results_df

,Training RMSE,Test RMSE
"Least Squares, Subset",1.353990,1.401749
"Least Squares, All",1.353990,1.401749
Ridge,1.355483,1.402178
LASSO,1.389815,1.438330


In [29]:
results.head()

,Index,avg_price_per_kg
0,1,10.510566
1,2,11.852054
2,3,10.019409
3,4,8.486529
4,5,8.017801


In [30]:
results1.head()

,Index,avg_price_per_kg
0,1,10.762041
1,2,11.927670
2,3,10.023420
3,4,8.724784
4,5,8.032517


In [31]:
results2.head()

,Index,Price
0,1,10.507788
1,2,11.523762
2,3,9.791005
3,4,8.512146
4,5,8.111765
